In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import os,errno
import sys
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import pandas as pd
import datetime as dt
from scipy.ndimage.measurements import label
from math import sin, cos, sqrt, atan2, radians
import geopy.distance
%matplotlib inline

dir2='/thorncroftlab_rit/ahenny/rain/'
dir1='/thorncroftlab_rit/ahenny/rain/US/ghcnd_all/'
dir='/thorncroftlab_rit/ahenny/rain/DISSERTATION_SCRIPTS_RESULTS/'

In [ ]:
ds=xr.open_dataset(dir+'extreme_days_ghcnd_99station_80area95.nc')
lats=ds['lats'].values.tolist()
lons=ds['lons'].values.tolist()
lons=[x+360. for x in lons]
dates=ds['dates'].values
dates_unique=list(set(dates))
dates_unique=pd.DatetimeIndex(dates_unique).sort_values()
stations=ds['stations'].values.tolist()
obs=ds['obs'].values.tolist()
years=[x.year for x in dates_unique]
print(dates_unique)

ds6=xr.open_dataset(dir+'IBTrACS.NA.v04r00.nc')
print(ds6.lat.description)
wind_tc=ds6.usa_wind
lat_tc=ds6.lat
lon_tc=ds6.lon
#nature_tc=ds6.nature
ds7=xr.open_dataset(dir+'ibtracs_na_time.nc')
years_tc=ds7.years
months_tc=ds7.months
days_tc=ds7.days
hours_tc=ds7.hours
nature_tc=ds7.nature#1 if tropical, 0 if not
print(nature_tc)
print(hours_tc)

#TC activity in the week preceding the extreme precipitation day

In [ ]:
ds4=xr.open_dataset(dir+'neusa_ep_days_stats_var95const_station95.nc')
ar_yesno4=ds4['ar_yesno'].values.tolist()
ivt_yesno4=ds4['ivt_yesno'].values.tolist()
tc_yesno4=ds4['tc_yesno'].values.tolist()
other_yesno4=ds4['other_yesno'].values.tolist()
tc_linked_ar_yesno4=ds4['tc_linked_ar_yesno'].values.tolist()
tc_linked_ivt_yesno4=ds4['tc_linked_ivt_yesno'].values.tolist()
tc_remnant_linked_ivt_yesno4=ds4['ivt_tc_remnants_yesno'].values.tolist()
tc_remnant_ar_combo_yesno4=ds4['ar_tc_remnant_combo_yesno'].values.tolist()
tc_remnant_linked_ar_yesno4=ds4['tc_remnant_linked_ar_yesno'].values.tolist()
tc_remnants_yesno4=ds4['tc_remnants_yesno'].values.tolist()
tc_ar_combo_yesno4=ds4['tc_ar_combo_yesno'].values.tolist()

#these dates are non-TC-dominant EP days that have been associated with TC influences in the preceding 3 days
dates_remote_tc_inf=[dt.datetime(1979,9,21,6),dt.datetime(1987,9,9,6),dt.datetime(1989,9,19,6),
                     dt.datetime(1996,9,15,6),dt.datetime(2002,9,16,6),
                     dt.datetime(2002,9,28,6),dt.datetime(2003,9,2,6),dt.datetime(2003,9,4,6),
                     dt.datetime(2005,9,1,6),dt.datetime(2005,10,7,6),dt.datetime(2005,10,8,6),
                     dt.datetime(2005,10,9,6),dt.datetime(2005,10,15,6),dt.datetime(2009,11,12,6),
                     dt.datetime(2010,10,1,6),dt.datetime(2011,9,7,6),dt.datetime(2011,9,8,6),
                     dt.datetime(2015,9,30,6),dt.datetime(2015,10,1,6),dt.datetime(2017,10,29,6),
                     dt.datetime(2017,10,30,6),dt.datetime(2018,9,9,6),dt.datetime(2018,9,10,6),
                     dt.datetime(2018,9,19,6)]
remote_tc_inf_yesno4=[]
for i in range(len(dates_unique)):
    if dates_unique[i] in dates_remote_tc_inf:
        remote_tc_inf_yesno4.append(1)
    else:
        remote_tc_inf_yesno4.append(0)
print(sum(remote_tc_inf_yesno4))

zipped_t1=list(zip(dates_unique,ar_yesno4))#ar-related = green
zipped_t2=list(zip(dates_unique,tc_linked_ar_yesno4))
zipped_t3=list(zip(dates_unique,tc_remnant_linked_ar_yesno4))
zipped_t4=list(zip(dates_unique,tc_yesno4))#tc-related = blue
zipped_t5=list(zip(dates_unique,tc_ar_combo_yesno4))
zipped_t6=list(zip(dates_unique,tc_remnant_ar_combo_yesno4))
zipped_t7=list(zip(dates_unique,tc_remnants_yesno4))
zipped_t8=list(zip(dates_unique,ivt_yesno4))#other IVT-related = grey
zipped_t9=list(zip(dates_unique,tc_linked_ivt_yesno4))
zipped_t10=list(zip(dates_unique,tc_remnant_linked_ivt_yesno4))
zipped_t11=list(zip(dates_unique,other_yesno4))#unspecified = brown

dates_t1=[x[0] for x in zipped_t1 if x[1]==1]
dates_t2=[x[0] for x in zipped_t2 if x[1]==1]
dates_t3=[x[0] for x in zipped_t3 if x[1]==1]
dates_t4=[x[0] for x in zipped_t4 if x[1]==1]
dates_t5=[x[0] for x in zipped_t5 if x[1]==1]
dates_t6=[x[0] for x in zipped_t6 if x[1]==1]
dates_t7=[x[0] for x in zipped_t7 if x[1]==1]
dates_t8=[x[0] for x in zipped_t8 if x[1]==1]
dates_t9=[x[0] for x in zipped_t9 if x[1]==1]
dates_t10=[x[0] for x in zipped_t10 if x[1]==1]
dates_t11=[x[0] for x in zipped_t11 if x[1]==1]
dates_t12=dates_remote_tc_inf

dates_ar=dates_t1+dates_t2+dates_t3
dates_tc=dates_t4+dates_t5+dates_t6+dates_t7
dates_other=dates_t8+dates_t9+dates_t10+dates_t11
dates_inf=dates_tc+dates_t2+dates_t3+dates_t9+dates_t10+dates_t12

dates_t1=pd.DatetimeIndex(dates_t1)
dates_t2=pd.DatetimeIndex(dates_t2)
dates_t3=pd.DatetimeIndex(dates_t3)
dates_t4=pd.DatetimeIndex(dates_t4)
dates_t5=pd.DatetimeIndex(dates_t5)
dates_t6=pd.DatetimeIndex(dates_t6)
dates_t7=pd.DatetimeIndex(dates_t7)
dates_t8=pd.DatetimeIndex(dates_t8)
dates_t9=pd.DatetimeIndex(dates_t9)
dates_t10=pd.DatetimeIndex(dates_t10)
dates_t11=pd.DatetimeIndex(dates_t11)
dates_t12=pd.DatetimeIndex(dates_t12)

dates_ar=pd.DatetimeIndex(dates_ar)
dates_tc=pd.DatetimeIndex(dates_tc)
dates_other=pd.DatetimeIndex(dates_other)
dates_inf=pd.DatetimeIndex(dates_inf)

years_t1=[x.year for x in dates_t1]
years_t2=[x.year for x in dates_t2]
years_t3=[x.year for x in dates_t3]
years_t4=[x.year for x in dates_t4]
years_t5=[x.year for x in dates_t5]
years_t6=[x.year for x in dates_t6]
years_t7=[x.year for x in dates_t7]
years_t8=[x.year for x in dates_t8]
years_t9=[x.year for x in dates_t9]
years_t10=[x.year for x in dates_t10]
years_t11=[x.year for x in dates_t11]
years_t12=[x.year for x in dates_t12]

years_tc1=[x.year for x in dates_tc]
years_ar=[x.year for x in dates_ar]
years_other=[x.year for x in dates_other]
years_inf=[x.year for x in dates_inf]

In [ ]:
#to draw grid lines on Lambert Conformal projection; 
#CREDIT ajdawson on GitHub https://gist.github.com/ajdawson/dd536f786741e987ae4e

from copy import copy
import shapely.geometry as sgeom
def find_side(ls, side):
    """
    Given a shapely LineString which is assumed to be rectangular, return the
    line corresponding to a given side of the rectangle.
    
    """
    minx, miny, maxx, maxy = ls.bounds
    points = {'left': [(minx, miny), (minx, maxy)],
              'right': [(maxx, miny), (maxx, maxy)],
              'bottom': [(minx, miny), (maxx, miny)],
              'top': [(minx, maxy), (maxx, maxy)],}
    return sgeom.LineString(points[side])


def lambert_xticks(ax, ticks):
    """Draw ticks on the bottom x-axis of a Lambert Conformal projection."""
    te = lambda xy: xy[0]
    lc = lambda t, n, b: np.vstack((np.zeros(n) + t, np.linspace(b[2], b[3], n))).T
    xticks, xticklabels = _lambert_ticks(ax, ticks, 'bottom', lc, te)
    ax.xaxis.tick_bottom()
    ax.set_xticks(xticks)
    ax.set_xticklabels([ax.xaxis.get_major_formatter()(xtick) for xtick in xticklabels])
    

def lambert_yticks(ax, ticks):
    """Draw ricks on the left y-axis of a Lamber Conformal projection."""
    te = lambda xy: xy[1]
    lc = lambda t, n, b: np.vstack((np.linspace(b[0], b[1], n), np.zeros(n) + t)).T
    yticks, yticklabels = _lambert_ticks(ax, ticks, 'left', lc, te)
    ax.yaxis.tick_left()
    ax.set_yticks(yticks)
    ax.set_yticklabels([ax.yaxis.get_major_formatter()(ytick) for ytick in yticklabels])
def _lambert_ticks(ax, ticks, tick_location, line_constructor, tick_extractor):
    """Get the tick locations and labels for an axis of a Lambert Conformal projection."""
    outline_patch = sgeom.LineString(ax.outline_patch.get_path().vertices.tolist())
    axis = find_side(outline_patch, tick_location)
    n_steps = 30
    extent = ax.get_extent(ccrs.PlateCarree())
    _ticks = []
    for t in ticks:
        xy = line_constructor(t, n_steps, extent)
        proj_xyz = ax.projection.transform_points(ccrs.Geodetic(), xy[:, 0], xy[:, 1])
        xyt = proj_xyz[..., :2]
        ls = sgeom.LineString(xyt.tolist())
        locs = axis.intersection(ls)
        if not locs:
            tick = [None]
        else:
            tick = tick_extractor(locs.xy)
        _ticks.append(tick[0])
    # Remove ticks that aren't visible:    
    ticklabels = copy(ticks)
    while True:
        try:
            index = _ticks.index(None)
        except ValueError:
            break
        _ticks.pop(index)
        ticklabels.pop(index)
    return _ticks, ticklabels

In [ ]:
month_dict={'1':'January','2':'February','3':'March','4':'April','5':'May','6':'June','7':'July','8':'August','9':'September','10':'October','11':'November','12':'December'}    
clon=-70
clat=35
#proj_map=ccrs.PlateCarree()
proj_map = ccrs.LambertConformal(central_longitude=clon, central_latitude=clat)
fig = plt.figure(figsize=(24,16))
ax=plt.subplot(1,1,1,projection=proj_map)

ds1=xr.open_dataset(dir+'station_numbers.nc')
lats1=ds1['lats']#use these lists for plotting
lons1=ds1['lons']+360.
stations1=ds1['stations']
thresholds=ds1['thresholds']/10.
zipped_thresh=list(zip(thresholds,lats1,lons1))

years_zipped=list(zip(years_tc1,dates_tc))
dates_tc_first=[x[1] for x in years_zipped if x[0]<=1999]
dates_tc_second=[x[1] for x in years_zipped if x[0]>=2000]
years_tc_first=[x[0] for x in years_zipped if x[0]<=1999]
years_tc_second=[x[0] for x in years_zipped if x[0]>=2000]

years_zipped1=list(zip(years_ar,dates_ar))
dates_ar_first=[x[1] for x in years_zipped1 if x[0]<=1999]
dates_ar_second=[x[1] for x in years_zipped1 if x[0]>=2000]
years_ar_first=[x[0] for x in years_zipped1 if x[0]<=1999]
years_ar_second=[x[0] for x in years_zipped1 if x[0]>=2000]

years_zipped2=list(zip(years_other,dates_other))
dates_other_first=[x[1] for x in years_zipped2 if x[0]<=1999]
dates_other_second=[x[1] for x in years_zipped2 if x[0]>=2000]
years_other_first=[x[0] for x in years_zipped2 if x[0]<=1999]
years_other_second=[x[0] for x in years_zipped2 if x[0]>=2000]

years_zipped3=list(zip(years_inf,dates_inf))
dates_inf_first=[x[1] for x in years_zipped3 if x[0]<=1999]
dates_inf_second=[x[1] for x in years_zipped3 if x[0]>=2000]
years_inf_first=[x[0] for x in years_zipped3 if x[0]<=1999]
years_inf_second=[x[0] for x in years_zipped3 if x[0]>=2000]

from matplotlib.collections import LineCollection
import matplotlib.patches as mpatches
from matplotlib import pyplot

print(dates_inf_first)
print(years_inf)
print(dates_inf)

zipped=list(zip(dates,lats,lons,obs))
index_list_date=[]
for d in range(len(dates_tc_first)):
#for d in range(2):
    print(date)
    date=dates_tc_first[d]#date is at 6Z automatically.  IVT goes from 6Z on day to 6Z on next day
    date_range=[date+dt.timedelta(hours=6*x) for x in range(4)]
    date_range=pd.DatetimeIndex(date_range).values#date range for IVT selection
    
    select_date=[x for x in zipped if x[0]==date]
    obs_extreme_list=[]
    lats_extreme_list=[]
    lons_extreme_list=[]
    for i in range(len(lats1)):
        lat=lats1[i]
        lon=lons1[i]
        select_station=[x for x in select_date if x[1]==lat and x[2]==lon]
        select_thresh=[x for x in zipped_thresh if x[1]==lat and x[2]==lon]
        if len(select_station)>1:
            print('ERROR')
        if len(select_station)>0:
            obs_1=select_station[0][-1]
            thresh=select_thresh[0][0].values
            if obs_1>=thresh:
                obs_extreme_list.append(obs_1)
                lats_extreme_list.append(lat)
                lons_extreme_list.append(lon)
    
    count=0
    year=date.year
    check_lat_tc=lat_tc.where(years_tc==year,drop=True)
    check_lon_tc=lon_tc.where(years_tc==year,drop=True)
    storm_indices=check_lat_tc.storm.values.tolist()
    
    index_list=[]
    print(date_range)
    for date in date_range:
        date=pd.to_datetime(date)
        month=date.month
        day=date.day
        hour=date.hour
        
        check_lat_tc1=check_lat_tc.where(months_tc==month,drop=True)
        check_lat_tc1=check_lat_tc1.where(days_tc==day,drop=True)
        check_lat_tc1=check_lat_tc1.where(hours_tc==hour,drop=True)
            
        check_lon_tc1=check_lon_tc.where(months_tc==month,drop=True)
        check_lon_tc1=check_lon_tc1.where(days_tc==day,drop=True)
        check_lon_tc1=check_lon_tc1.where(hours_tc==hour,drop=True) 
        
        check_lat_tc_list=[]
        check_lon_tc_list=[]
        for l in check_lat_tc1.storm.values:
            print(l)
            min_distance_tc=2000.
            check_lat_tc_values=check_lat_tc1.sel(storm=l).values.tolist()
            check_lat_tc_values=[x for x in check_lat_tc_values if x>0]

            check_lon_tc_values=check_lon_tc1.sel(storm=l).values.tolist()
            check_lon_tc_values=[x for x in check_lon_tc_values if x>-360]
            
            if len(check_lon_tc_values)>0:
                
                index_list.append(l)
                
                lon_tc_single=check_lon_tc_values[0]
                lat_tc_single=check_lat_tc_values[0]
                for j in range(len(lats_extreme_list)):
                    lat_rain=lats_extreme_list[j]
                    lon_rain=lons_extreme_list[j]
                    distance=geopy.distance.distance((lat_tc_single,lon_tc_single),(lat_rain,lon_rain)).km
                    if distance<min_distance_tc:
                        min_distance_tc=distance
            #if min_distance_tc<=500:
            #    print('YES')
            #    index_list.append(l)
    index_list=list(set(index_list))
    index_list_date.append(index_list)
        
print(index_list_date)
#sys.exit()
print('A')
for d in range(len(dates_tc_first)):
#for d in range(len(dates_unique)):
    print(d)
    date=dates_tc_first[d]#date is at 6Z automatically. 
    date_day_end=date+dt.timedelta(hours=24)
    date_start=date_day_end+dt.timedelta(days=-7)
    date_range=[date_start+dt.timedelta(hours=6*x) for x in range(28)]
    date_range=pd.DatetimeIndex(date_range).values
    year=years_tc_first[d]
    index_list=index_list_date[d]
    
    check_lat_tc=lat_tc.where(years_tc==year,drop=True)
    check_lon_tc=lon_tc.where(years_tc==year,drop=True)
    check_months_tc=months_tc.where(years_tc==year,drop=True)
    check_days_tc=days_tc.where(years_tc==year,drop=True)
    check_hours_tc=hours_tc.where(years_tc==year,drop=True)
    check_nature_tc=nature_tc.where(years_tc==year,drop=True)
    check_wind_tc=wind_tc.where(years_tc==year,drop=True)
    segs=[]
    print(index_list)
    for i in check_lat_tc.storm.values:
        if i in index_list:
            storm_lat=check_lat_tc.sel(storm=i).values.tolist()
            storm_lon=check_lon_tc.sel(storm=i).values.tolist()
            storm_month=check_months_tc.sel(storm=i).values.tolist()
            storm_day=check_days_tc.sel(storm=i).values.tolist()
            storm_hour=check_hours_tc.sel(storm=i).values.tolist()
            storm_wind=check_wind_tc.sel(storm=i).values.tolist()
            storm_nature=check_nature_tc.sel(storm=i).values.tolist()
            storm_month=[x for x in storm_month if x>=0]
            storm_day=[x for x in storm_day if x>=0]
            storm_hour=[x for x in storm_hour if x>=0]
            storm_date=[pd.to_datetime(dt.datetime(year,int(x),int(y),int(z))) for x,y,z in zip(storm_month,storm_day,storm_hour)]
            storm_wind=[x for x in storm_wind if x>=0]
            zipped_latlon=list(zip(storm_lat,storm_lon,storm_date,storm_wind,storm_nature))
            #select_week=[x for x in zipped_latlon if x[2] in date_range]
            select_lat=[x[0] for x in select_week]
            select_lon=[x[1] for x in select_week]
            select_wind=[x[3] for x in select_week]
            select_nature=[x[4] for x in select_week]
            if len(select_week)>0:
                for j in range(len(select_week)-1):
                    lat_1=select_lat[j]
                    lon_1=select_lon[j]
                    lat_2=select_lat[j+1]
                    lon_2=select_lon[j+1]
                    starting_wind=select_wind[j]#in kts
                    starting_nature=select_nature[j]
                    #now choose color based on category
                    if starting_nature==1:
                        if starting_wind<=33:
                            color='b'
                        if 34<=starting_wind<=63:
                            color='g'
                        if 64<=starting_wind<=82:
                            color='yellow'
                        if 83<=starting_wind<=95:
                            color='orange'
                        if 96<=starting_wind<=112:
                            color='r'
                        if 113<=starting_wind<=136:
                            color='darkred'
                        if starting_wind>=137:
                            color='m'
                    else:
                        color='k'

                    seg=[[lon_1,lat_1],[lon_2,lat_2]]
                    ax.plot([lon_1,lon_2],[lat_1,lat_2],transform=ccrs.PlateCarree(),linewidth=3.0,color=color)
                    segs.append(seg)

ax.coastlines(resolution='10m')
ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
ax.add_feature(cfeature.LAKES.with_scale('50m'))
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
ax.set_extent([-95,-48,10,50],crs=ccrs.PlateCarree())

# *must* call draw in order to get the axis boundary used to add ticks:
fig.canvas.draw()

# Define gridline locations and draw the lines using cartopy's built-in gridliner:
xticks = [-135,-130,-125,-120,-115,-110,-105,-100,-95,-90,-85,-80,-75,-70,-65,-60,-55,-50,-45,-40,-35,-30,-25,-20]
yticks = [5,10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
ax.gridlines(xlocs=xticks, ylocs=yticks,alpha=0.5)
ax.tick_params(labelsize=25)
# Label the end-points of the gridlines using the custom tick makers:
ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)
lambert_xticks(ax, xticks)
lambert_yticks(ax, yticks)
ax.set_title('TC-dominant weather types: 1979-1999',fontsize=36)#2000 - 2019; 1979-1999

patch1 = mpatches.Patch(color='b', alpha=1,label='TD')
patch2 = mpatches.Patch(color='g', alpha=1,label='TS')
patch3 = mpatches.Patch(color='yellow', alpha=1,label='Cat 1')
patch4 = mpatches.Patch(color='orange', alpha=1,label='Cat 2')
patch5 = mpatches.Patch(color='r', alpha=1,label='Cat 3')
patch6 = mpatches.Patch(color='darkred', alpha=1,label='Cat 4')
patch7 = mpatches.Patch(color='m', alpha=1,label='Cat 5')
patch8 = mpatches.Patch(color='k', alpha=1,label='Non-tropical')

legend1=pyplot.legend(handles=[patch1,patch2,patch3,patch4,patch5,patch6,patch7,patch8],loc='upper left',prop={'size':19},fancybox=True, framealpha=1.0)
pyplot.gca().add_artist(legend1)

plt.show()

In [ ]:
fig.savefig(dir+'neusa_tc_halves_33.png')

In [ ]:
print(dates_ar_first)